In [1]:
from json import load
import pandas as pd

In [2]:
with open("maestro_sample_log.json") as jsonfile:
    data = load(jsonfile)
    
jsonfile.close()

In [3]:
processed = []
for i in data:
    sample = data[i]
    name = sample['name']
    storage_slot = sample['storage_slot']['slot']
    storage_tray = sample['storage_slot']['tray']
    substrate = sample['substrate']
    worklist = sample['worklist'] # need to deal with this somehow
    hotplate = sample['hotplate_slot']['hotplate']
    hoteplate_slot = sample['hotplate_slot']['slot']
    processed.append([name, storage_slot, storage_tray, substrate, worklist, hotplate, hoteplate_slot])

In [4]:
df = pd.DataFrame(processed, columns = ['name', 
                                        'storage_slot', 
                                        'storage_tray', 
                                        'substrate', 
                                        'worklist', 
                                        'hotplate', 
                                        'hoteplate_slot'])
df.head(5)

,name,storage_slot,storage_tray,substrate,worklist,hotplate,hoteplate_slot
0,sample0,A1,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,B3
1,sample1,A2,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,B4
2,sample10,C1,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,D2
3,sample11,C2,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,D3
4,sample12,C3,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,C3


In [5]:
# for step in data['sample27']['worklist']:
#     print(step)

In [6]:
def flatten_characterization(worklist):
    result = {}
    for task in worklist:
        if 'details' in task and 'characterization_tasks' in task['details']:
            for image_type in task['details']['characterization_tasks']:
                img_station =  image_type['station']
                if 'exposure_times' in image_type['details']:
                    result['{}_exposure_time'.format(img_station)] = image_type['details']['exposure_times']
                elif 'exposure_time' in image_type['details']:
                    result['{}_exposure_time'.format(img_station)] = image_type['details']['exposure_time']
                if 'num_scans' in image_type['details']:
                    result['{}_num_scan'.format(img_station)] = image_type['details']['num_scans']
                elif 'num_scan' in image_type['details']:
                    result['{}_num_scan'.format(img_station)] = image_type['details']['num_scan']
                result['{}_duration'.format(img_station)] = image_type['duration']
        #         result['{}_name'.format(img_station)] = task['name']
                result['{}_position'.format(img_station)] = image_type['position']
    return result

# def flatten_drops(worklist):
#     """some drops have two steps, how should we deal with it?"""
#     result = {}
#     i = 1
#     for task in worklist:
#         if 'details' in task and 'drops' in task['details']:
#             for drop_task in task['details']['drops']:
#                 result['drop{}_air_gap'.format(i)] = drop_task['air_gap']
#                 result['drop{}_blow_out'.format(i)] = drop_task['blow_out']
#                 result['drop{}_height'.format(i)] = drop_task['height']
#                 result['drop{}_pre_mix'.format(i)] = drop_task['pre_mix']
#                 result['drop{}_rate'.format(i)] = drop_task['rate']
#                 result['drop{}_reuse_tip'.format(i)] = drop_task['reuse_tip']
#                 result['drop{}_slow_retract'.format(i)] = drop_task['slow_retract']
#                 result['drop{}_slow_travel'.format(i)] = drop_task['slow_travel']
# #                 result['drop{}_solution'.format(i)] = task['solution']
#                 result['drop{}_solution_molarity'.format(i)] = drop_task['solution']['molarity']
#                 result['drop{}_solution_solutes'.format(i)] = drop_task['solution']['solutes']
#                 result['drop{}_solution_solvent'.format(i)] = drop_task['solution']['solvent']
#                 result['drop{}_solution_well_labware'.format(i)] = drop_task['solution']['well']['labware']
#                 result['drop{}_solution_well'.format(i)] = drop_task['solution']['well']['well']
#                 result['drop{}_time'.format(i)] = drop_task['time']
#                 result['drop{}_touch_tip'.format(i)] = drop_task['touch_tip']
#                 result['drop{}_volume'.format(i)] = drop_task['volume']

#             result['drop{}_start_times'.format(i)] = task['details']['start_times']
#             result['drop{}_duration'.format(i)] = task['details']['duration']
#             result['drop{}_steps'.format(i)] = task['details']['steps']
                
#             result['drop{}_id'.format(i)] = task['id']
#             result['drop{}_name'.format(i)] = task['name']
#             result['drop{}_precedent'.format(i)] = task['precedent']
#             result['drop{}_sample'.format(i)] = task['sample']
#             result['drop{}_start'.format(i)] = task['start']
#             result['drop{}_start_actual'.format(i)] = task['start_actual']
#             result['drop{}_liquidhandler_timings'.format(i)] = task['liquidhandler_timings']
#             result['drop{}_spincoater_log'.format(i)] = task['spincoater_log']
#             result['drop{}_headstart'.format(i)] = task['headstart']
#             result['drop{}_finish_actual'.format(i)] = task['finish_actual']
#             i += 1
#     return result

In [10]:
processed = []
for i in data:
    sample = data[i]
    worklist = sample['worklist'] # need to deal with this somehow
    cleaned_dict = {'name': sample['name'], 
                   'storage_slot':sample['storage_slot']['slot'], 
                   'storage_tray':sample['storage_slot']['tray'], 
                   'substrate':sample['substrate'], 
                   'worklist': worklist, 
                   'hotplate':hotplate, 
                   'hotplate_slot': sample['hotplate_slot']['slot']}
    
    characterization_img = flatten_characterization(worklist)
#     drop_step = flatten_drops(worklist)
#     processed.append({**cleaned_dict, **characterization_img, **drop_step})
    processed.append({**cleaned_dict, **characterization_img})

In [11]:
df = pd.DataFrame(processed)
df.head()

,name,storage_slot,storage_tray,substrate,worklist,hotplate,hotplate_slot,pl_red_exposure_time,pl_red_num_scan,pl_red_duration,...,transmission_exposure_time,transmission_num_scan,transmission_duration,transmission_position,darkfield_exposure_time,darkfield_duration,darkfield_position,brightfield_exposure_time,brightfield_duration,brightfield_position
0,sample0,A1,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,B3,"[0.1, 5, 20]",1,25.32,...,"[0.02, 0.05, 0.2, 1, 5, 15]",2,46.54,286,0.05,2.5,74.2,0.05,0.05,164.4
1,sample1,A2,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,B4,"[0.1, 5, 20]",1,25.32,...,"[0.02, 0.05, 0.2, 1, 5, 15]",2,46.54,286,0.05,2.5,74.2,0.05,0.05,164.4
2,sample10,C1,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,D2,"[0.1, 5, 20]",1,25.32,...,"[0.02, 0.05, 0.2, 1, 5, 15]",2,46.54,286,0.05,2.5,74.2,0.05,0.05,164.4
3,sample11,C2,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,D3,"[0.1, 5, 20]",1,25.32,...,"[0.02, 0.05, 0.2, 1, 5, 15]",2,46.54,286,0.05,2.5,74.2,0.05,0.05,164.4
4,sample12,C3,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,C3,"[0.1, 5, 20]",1,25.32,...,"[0.02, 0.05, 0.2, 1, 5, 15]",2,46.54,286,0.05,2.5,74.2,0.05,0.05,164.4


In [12]:
def flatten_samples(sample):
    result = {}
    result['sample_num'] = sample['name']
    result['step_count'] = len(sample['worklist'])
    result['tray_pos'] = sample['storage_slot']['tray']
    result['tray_slot'] = sample['storage_slot']['slot']
    return result

In [48]:
def flatten_steps(sample):
    result = []
    worklist = sample['worklist']
    for i in range(0, len(worklist)):
        step = {}
        step['sample_num'] = sample['name']
        step['step_num'] = i + 1
        step['step_type'] = list(worklist[i]['details'].keys())[0] # dictionary order is not correct fix later
        step['time_scheduled'] = 0 # fix later
        step['time_start_nominal'] = worklist[i]['start']
        step['time_start'] = worklist[i]['start_actual']
        step['time_stop'] = worklist[i]['finish_actual']
        result.append(step)
    return result

# show the first two just to see
flatten_steps(data['sample11'])[:2]

[{'sample_num': 'sample11',
  'step_num': 1,
  'step_type': 'destination',
  'time_scheduled': 0,
  'time_start_nominal': 2453,
  'time_start': 2453.608620405197,
  'time_stop': 2481.0518457889557},
 {'sample_num': 'sample11',
  'step_num': 2,
  'step_type': 'drops',
  'time_scheduled': 0,
  'time_start_nominal': 2486,
  'time_start': 2485.9941008090973,
  'time_stop': 2576.6879074573517}]

In [45]:
# def flatten_drops(sample):
#     result = []
#     worklist = sample['worklist']
#     for i in range(0, len(worklist)):
#         if 'details' in task and 'drops' in task['details']:
#             drop = {}
#             drop['sample_num'] = sample['name']
#             drop['step_num'] = i + 1
#             # worklist[i]['details']['drops'] is a list of size 1, can it be bigger? 
#             for j in worklist:
#                 if j['solution']
#             result.append(drop)
#     return result
# data['sample10']['worklist'][1]['details']['drops']

In [60]:
def flatten_hotplate(sample):
    result = []
    worklist = sample['worklist']
    for i in range(0, len(worklist)):
        if 'hotplate' in worklist[i]['details']:        
            hotplate = {}
            hotplate['sample_num'] = sample['name']
            hotplate['step_num'] = i + 1
            hotplate['plate_num'] = worklist[i]['details']['hotplate']
            hotplate['temperature'] = worklist[i]['details']['temperature']
            hotplate['duration'] = worklist[i]['details']['duration']
            result.append(hotplate)
    return result
flatten_hotplate(data['sample1'])

[{'sample_num': 'sample1',
  'step_num': 4,
  'plate_num': 'Hotplate1',
  'temperature': 110,
  'duration': 600},
 {'sample_num': 'sample1',
  'step_num': 10,
  'plate_num': 'Hotplate1',
  'temperature': 110,
  'duration': 1800}]

In [70]:
def flatten_characterization(sample):
    result = {}
    worklist = sample['worklist']
    for i in range(0, len(worklist)):
        if 'characterization_tasks' in worklist[i]['details']:        
            for image_type in worklist[i]['details']['characterization_tasks']:
                img_station =  image_type['station']
                if 'exposure_times' in image_type['details']:
                    result['{}_exposure_time'.format(img_station)] = image_type['details']['exposure_times']
                elif 'exposure_time' in image_type['details']:
                    result['{}_exposure_time'.format(img_station)] = image_type['details']['exposure_time']
                if 'num_scans' in image_type['details']:
                    result['{}_num_scan'.format(img_station)] = image_type['details']['num_scans']
                elif 'num_scan' in image_type['details']:
                    result['{}_num_scan'.format(img_station)] = image_type['details']['num_scan']
                if 'num_frames' in image_type['details']:
                    result['{}_num_frame'.format(img_station)] = image_type['details']['num_frames']
                elif 'num_frame' in image_type['details']:
                    result['{}_num_frame'.format(img_station)] = image_type['details']['num_frame']
                result['{}_duration'.format(img_station)] = image_type['duration']
        #         result['{}_name'.format(img_station)] = task['name']
                result['{}_position'.format(img_station)] = image_type['position']
    return result

flatten_characterization(data['sample11'])

{'pl_red_exposure_time': [0.1, 5, 20],
 'pl_red_num_scan': 1,
 'pl_red_duration': 25.32,
 'pl_red_position': 286,
 'transmission_exposure_time': [0.02, 0.05, 0.2, 1, 5, 15],
 'transmission_num_scan': 2,
 'transmission_duration': 46.54,
 'transmission_position': 286,
 'darkfield_exposure_time': 0.05,
 'darkfield_num_frame': 50,
 'darkfield_duration': 2.5,
 'darkfield_position': 74.2,
 'brightfield_exposure_time': 0.05,
 'brightfield_num_frame': 1,
 'brightfield_duration': 0.05,
 'brightfield_position': 164.4}